### 문제 2-1 : 콤마 구분 리스트 파서 활용

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [6]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
너는 한국 여행 전문가야. 사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력해.
입력된 분야: {interest}
{format_instructions}
    """.strip(),
    input_variables=["interest"],
    partial_variables={"format_instructions": format_instructions},
)

user_interest = input().strip()

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1", 
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

formatted_prompt = prompt.format(interest=user_interest)
response = model.predict(formatted_prompt)
result = output_parser.parse(response)

print(result)


['서울의 N서울타워 야경', '부산의 광안리 해수욕장', '인천의 차이나타운', '경주의 황남빵', '전주 한옥마을의 전통 음식점']


### 문제 2-2 : 영화 리뷰 감정 분석기

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum

class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

template = """
당신은 감정 분석 전문가입니다.
사용자가 입력한 영화 리뷰를 읽고 감정을 판단하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 보통
input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 보통'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 감정 분석 전문가입니다.\n사용자가 입력한 영화 리뷰를 읽고 감정을 판단하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요\n2. 다른 설명이나 부가 설명을 추가하지 마세요\n3. 이모지나 특수문자도 포함하지 마세요\n4. 오직 하나의 단어만 출력하세요\n\n답변:'), additional_kwargs={})]


In [11]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1", 
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

print("모델 및 파서 설정 완료")

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

모델 및 파서 설정 완료
테스트할 텍스트 3개 준비 완료


In [14]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else enumParser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 

=== 결과 요약 ===
성공: 3/3 (100.0%)
실패: 0/3


### 문제 2-3: 학생 정보 구조화 시스템

In [15]:
# poetry add pydantic
# %pip install pydantic 

from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

In [21]:
# 출력 구조를 정의하는 Pydantic 모델
class StudentInfo(BaseModel):
    name: str = Field(..., description="학생의 이름")
    age: int = Field(..., description="학생의 나이")
    major: str = Field(..., description="학생의 전공")
    hobbies: list[str] = Field(..., description="학생의 취미 목록")
    goal: str = Field(..., description="학생의 목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 학생의 이름(name), 나이(age), 전공(major), 취미 리스트(hobbies), 목표(goal)를
영어 필드명으로 JSON 형식으로 출력하세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "학생의 이름", "title": "Name", "type": "string"}, "age": {"description": "학생의 나이", "title": "Age", "type": "integer"}, "major": {"description": "학생의 전공", "title": "Major", "type": "string"}, "hobbies": {"description": "학생의 취미 목록", "items": {"type": "string"}, "title": "Hobbies", "type": "array"}, "goal": {"description": "학생의 목표", "title": "Goal", "type": "string"}}, "required": ["name", "age", "major", "hobbies", "goal

In [23]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
output = prompt.format_messages(query=query, format_instructions=format_instructions)
response = model(output)
student_info = parser.parse(response.content)

print(student_info.model_dump_json(indent=4))

{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}
